In [13]:
import panel as pn
import pandas as pd
import requests
import hvplot.pandas
import holoviews as hv
from holoviews import opts
from bokeh.models import HoverTool, PanTool, BoxZoomTool, WheelZoomTool, ResetTool
import numpy as np

def get_coingecko_coins():
    url = 'https://api.coingecko.com/api/v3/coins/list'
    response = requests.get(url)
    data = response.json()
    coin_dict = {coin['symbol'].upper(): coin['id'] for coin in data}
    return coin_dict

coin_dict = get_coingecko_coins()

def get_historical_data(coin_id, vs_currency, days):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency={vs_currency}&days={days}&interval=daily'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame({'timestamp': [x[0] for x in data['prices']], 'price': [x[1] for x in data['prices']]})
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df['returns'] = df['price'].pct_change() 
    return df.dropna()

def calculate_total_return(df):
    return (df['price'].iloc[-1] / df['price'].iloc[0]) - 1

benchmark_select_widget = pn.widgets.AutocompleteInput(name='Benchmark Asset:', options=list(coin_dict.keys()), value='None')
coin_select_widgets = [pn.widgets.AutocompleteInput(name=f'Asset {i+1}:', options=list(coin_dict.keys()), value='None') for i in range(5)]
timeframe_select_widget = pn.widgets.Select(name='Timeframe:', options=['24h', '7d', '30d', '60d', '90d', 'max'], value='max')
metric_checkboxes = [pn.widgets.Checkbox(name=metric, value=False) for metric in ['Total return', 'Standard deviation', 'Sharpe ratio', 'Beta', 'Sortino ratio']]
visualise_button = pn.widgets.Button(name='Visualise Performance', button_type='primary')
risk_free_rate_input = pn.widgets.FloatInput(name='Risk-free rate:', value=0.01, step=0.01)
plot_colors = ['orange', 'blue', 'green', 'red', 'purple', 'black']

def visualise_portfolio(event):
    global coin_dict
    benchmark_id = coin_dict[benchmark_select_widget.value.upper()]
    coin_ids = [coin_dict[coin_select.value.upper()] for coin_select in coin_select_widgets if coin_select.value]
    timeframe = timeframe_select_widget.value
    selected_metrics = [checkbox.name for checkbox in metric_checkboxes if checkbox.value]
    risk_free_rate = risk_free_rate_input.value
    plots = []
    benchmark_df = get_historical_data(benchmark_id, 'usd', timeframe)
    if 'Standard deviation' in selected_metrics:
        benchmark_std = benchmark_df['returns'].std()
        print(f"Benchmark standard deviation: {benchmark_std}")
    if 'Sharpe ratio' in selected_metrics:
        benchmark_sharpe = (benchmark_df['returns'].mean() - risk_free_rate) / benchmark_df['returns'].std()
        print(f"Benchmark Sharpe ratio: {benchmark_sharpe}")
    if 'Total return' in selected_metrics:
        benchmark_total_return = calculate_total_return(benchmark_df)
        print(f"Benchmark total return: {benchmark_total_return}")

    hover = HoverTool(
        tooltips=[
            ('date', '$x{%F}'),
            ('price', '@price'),
        ],
        formatters={
            '$x': 'datetime',
        },
        mode='vline'
    )
    
    benchmark_plot = benchmark_df.hvplot(
        y='price',
        title='Portfolio',
        width=1315,
        height=830,
        color=plot_colors[0],
        label=benchmark_select_widget.value.upper()
    ).opts(
        bgcolor='#D3D3D3',
        tools=[PanTool(), BoxZoomTool(), WheelZoomTool(), ResetTool(), hover],
        show_grid=True,
        gridstyle=dict(
            grid_line_color='white'
        )
    )
    plots.append(benchmark_plot)

    for i, coin_id in enumerate(coin_ids):
        df = get_historical_data(coin_id, 'usd', timeframe)

        if 'Standard deviation' in selected_metrics:
            std = df['returns'].std()
            print(f"Asset {i+1} standard deviation: {std}")
        if 'Sharpe ratio' in selected_metrics:
            sharpe = (df['returns'].mean() - risk_free_rate) / df['returns'].std()
            print(f"Asset {i+1} Sharpe ratio: {sharpe}")
        if 'Total return' in selected_metrics:
            total_return = calculate_total_return(df)
            print(f"Asset {i+1} total return: {total_return}")

        plot = df.hvplot(
            y='price',
            width=1315,
            height=830,
            color=plot_colors[i+1],
            label=coin_select_widgets[i].value.upper()
        ).opts(
            bgcolor='#D3D3D3',
            tools=[PanTool(), BoxZoomTool(), WheelZoomTool(), ResetTool(), hover],
            show_grid=True,
            gridstyle=dict(
                grid_line_color='white'
            )
        )
        plots.append(plot)

    overlay_plot = hv.Overlay(plots).opts(legend_position='top_right')

    if selected_metrics:
        metrics_df = pd.DataFrame(index=coin_ids + [benchmark_id], columns=selected_metrics)

        for i, coin_id in enumerate(coin_ids + [benchmark_id]):
            df = get_historical_data(coin_id, 'usd', timeframe)
            for metric in selected_metrics:
                if metric == 'Standard deviation':
                    metrics_df.at[coin_id, 'Standard deviation'] = df['returns'].std()
                elif metric == 'Sharpe ratio':
                    metrics_df.at[coin_id, 'Sharpe ratio'] = (df['returns'].mean() - risk_free_rate) / df['returns'].std()
                elif metric == 'Total return':
                    metrics_df.at[coin_id, 'Total return'] = calculate_total_return(df)

        metrics_table = hv.Table(metrics_df, label='Portfolio Metrics - Beta & Sortino Ratio Coming Soon')

    dashboard.clear()
    dashboard.extend([benchmark_select_widget, timeframe_select_widget] + coin_select_widgets + metric_checkboxes + [visualise_button, pn.Row(overlay_plot, metrics_table)])

visualise_button.on_click(visualise_portfolio)

dashboard = pn.Column("Crypto Portfolio Performance Analyser", benchmark_select_widget, timeframe_select_widget, risk_free_rate_input, *coin_select_widgets, *metric_checkboxes, visualise_button)

dashboard.show()








Launching server at http://localhost:58271


Benchmark standard deviation: 0.10162971550003948
Benchmark Sharpe ratio: 0.04907175166790602
Benchmark total return: 327.6552308407378
Asset 1 standard deviation: 0.07488989252480747
Asset 1 Sharpe ratio: 0.04931567155125513
Asset 1 total return: 74.64423638477152
Asset 2 standard deviation: 0.11151925602972948
Asset 2 Sharpe ratio: 0.04604392525208457
Asset 2 total return: 34.184666030037334
Asset 3 standard deviation: 0.08394089218279224
Asset 3 Sharpe ratio: 0.05336377926179644
Asset 3 total return: 8.657210198103497
Asset 4 standard deviation: 0.0700656796614257
Asset 4 Sharpe ratio: 0.010717769599808464
Asset 4 total return: -0.9481704871029092
Asset 5 standard deviation: 0.0752498898802526
Asset 5 Sharpe ratio: 0.02597759841419301
Asset 5 total return: -0.3422085162618448
